In [1]:
import os
from pathlib import Path
from time import strftime

import mlflow
import pandas as pd
import tensorflow as tf

from utils.utils import (get_dataset, get_tokenizer, set_seed, tensorboard,
                         tracking)

2023-08-09 04:10:07.508074: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-09 04:10:07.542095: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-09 04:10:07.543260: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-09 04:10:08.560092: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
"""doc
"""

import re
import string

import hydra
import nltk
import polars as pl
from nltk.corpus import stopwords
from omegaconf import DictConfig

from utils.logger import logger

nltk.download("stopwords")


def clean_text(text_row):
    """performs preprocessing steps on each text row removing numbers,
    stopwords, punctuation and any symbols


    Returns
    -------
    clean_text : row
        A cleaned and preprocessed text
    """

    text_row = text_row.lower()
    text_row = re.sub("<[^>]*>", "", text_row)
    text_row = re.sub(r"[^a-zA-Z\s]", "", text_row)
    stop_words = set(stopwords.words("english"))
    text_row = [
        word
        for word in text_row.split()
        if word not in stop_words and word not in string.punctuation
    ]
    text_cleaned = " ".join(word for word in text_row)
    return text_cleaned


def label_encoder(target_df):
    """performs label encoding for target label


    Returns
    -------
    label : int
        return either 0 for normal or 1 for abnormal
    """

    if target_df == "normal":
        label = 0
    else:
        label = 1
    return label


def preprocess_and_encode(file_path, save_path):
    """_summary_

    Parameters
    ----------
    file_path : _type_
        _description_
    save_path : _type_
        _description_
    """
    dataframe = pl.read_parquet(file_path)
    dataframe = dataframe.with_columns(
        pl.col("Target").apply(label_encoder, return_dtype=pl.Int32)
    )
    dataframe = dataframe.with_columns(pl.col("Log").apply(clean_text))
    dataframe.write_parquet(file=save_path, compression="gzip")

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
os.chdir('../')
%pwd

'/workspace/log_anomaly'

In [4]:
set_seed()

Random seed set as 42


In [5]:
loss = tf.keras.losses.BinaryCrossentropy()
optim = tf.keras.optimizers.Adadelta(learning_rate=0.001)

In [6]:
from models import GRU

model_name ='New_Testing'

In [7]:
data_path = 'development/dev.gzip'

preprocess_and_encode(data_path, 'development/clean_test.gzip')

In [8]:
clean_data = 'development/clean_test.gzip'
dataset = get_dataset(file_path=clean_data,shuffle= True)
for sample in dataset.take(1):
    print(sample)

(<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'rmncju rmncju ras kernel info ce sym xef mask x',
       b'rmnccju rmnccju ras kernel info generating core'], dtype=object)>, <tf.Tensor: shape=(2,), dtype=int32, numpy=array([0, 0], dtype=int32)>)


2023-08-09 04:10:10.876700: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [9]:
tokenizer, vocab_size = get_tokenizer(dataset)
vocab_size

420

In [10]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return tokenizer(text), label
final_dataset = dataset.map(vectorize_text)

In [17]:
for sample in final_dataset.take(1):
    print(sample)

(<tf.Tensor: shape=(2, 20), dtype=int64, numpy=
array([[  2,   2,   3,   4,   5,  29,  30, 271,  28,  23,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  2,   2,   3,   4,   5,   7,   6,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0]])>, <tf.Tensor: shape=(2,), dtype=int32, numpy=array([0, 0], dtype=int32)>)


In [12]:
model = GRU.build_model(embed_dim=10, vocab_size=vocab_size,pad= 'valid')

In [13]:
dir = tensorboard(model_name)
model.compile(loss=loss, optimizer=optim, metrics= ['f1_score'])

In [14]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=dir)

In [16]:
model.fit(final_dataset, validation_split=0.2, callbacks=[tensorboard_cb])

ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'tensorflow.python.data.ops.map_op._MapDataset'>]